In [1]:
import datetime
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler

from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import ADASYN

from scipy.stats import loguniform, uniform, randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

c:\Python39\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (5.0.0)/charset_normalizer (2.0.6) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
nb_distribution = {'alpha':loguniform(0.001, 100)}

Import data

In [3]:
X_train_3perc_lag3 = pd.read_excel("data/model_inputs2/x_train_3perc_lag3.xlsx").rename(columns={'Unnamed: 0':'date'}).set_index('date')
X_test_3perc_lag3 = pd.read_excel("data/model_inputs2/x_test_3perc_lag3.xlsx").rename(columns={'Unnamed: 0':'date'}).set_index('date')
y_train_3perc_lag3 = pd.read_excel("data/model_inputs2/y_train_3perc_lag3.xlsx").rename(columns={'Unnamed: 0':'date'}).set_index('date')
y_test_3perc_lag3 = pd.read_excel("data/model_inputs2/y_test_3perc_lag3.xlsx").rename(columns={'Unnamed: 0':'date'}).set_index('date')

X_train_3perc_lag7 = pd.read_excel("data/model_inputs2/x_train_3perc_lag7.xlsx").rename(columns={'Unnamed: 0':'date'}).set_index('date')
X_test_3perc_lag7 = pd.read_excel("data/model_inputs2/x_test_3perc_lag7.xlsx").rename(columns={'Unnamed: 0':'date'}).set_index('date')
y_train_3perc_lag7 = pd.read_excel("data/model_inputs2/y_train_3perc_lag7.xlsx").rename(columns={'Unnamed: 0':'date'}).set_index('date')
y_test_3perc_lag7 = pd.read_excel("data/model_inputs2/y_test_3perc_lag7.xlsx").rename(columns={'Unnamed: 0':'date'}).set_index('date')

X_train_5perc_lag3 = pd.read_excel("data/model_inputs2/x_train_5perc_lag3.xlsx").rename(columns={'Unnamed: 0':'date'}).set_index('date')
X_test_5perc_lag3 = pd.read_excel("data/model_inputs2/x_test_5perc_lag3.xlsx").rename(columns={'Unnamed: 0':'date'}).set_index('date')
y_train_5perc_lag3 = pd.read_excel("data/model_inputs2/y_train_5perc_lag3.xlsx").rename(columns={'Unnamed: 0':'date'}).set_index('date')
y_test_5perc_lag3 = pd.read_excel("data/model_inputs2/y_test_5perc_lag3.xlsx").rename(columns={'Unnamed: 0':'date'}).set_index('date')

X_train_5perc_lag7 = pd.read_excel("data/model_inputs2/x_train_5perc_lag7.xlsx").rename(columns={'Unnamed: 0':'date'}).set_index('date')
X_test_5perc_lag7 = pd.read_excel("data/model_inputs2/x_test_5perc_lag7.xlsx").rename(columns={'Unnamed: 0':'date'}).set_index('date')
y_train_5perc_lag7 = pd.read_excel("data/model_inputs2/y_train_5perc_lag7.xlsx").rename(columns={'Unnamed: 0':'date'}).set_index('date')
y_test_5perc_lag7 = pd.read_excel("data/model_inputs2/y_test_5perc_lag7.xlsx").rename(columns={'Unnamed: 0':'date'}).set_index('date')

Oversampling methods

In [4]:
def smote(X_train, y_train):
    sm = SMOTE(sampling_strategy='not majority')
    X_train_oversampled, y_train_oversampled = sm.fit_resample(X_train, y_train)
    return X_train_oversampled, y_train_oversampled

def random_oversampler(X_train, y_train):
    oversample = RandomOverSampler(sampling_strategy='not majority')
    X_over, y_over = oversample.fit_resample(X_train, y_train)
    return X_over, y_over

def adasyn(X_train, y_train):
    ada = ADASYN(sampling_strategy = 'not majority')
    X_resampled, y_resampled = ADASYN().fit_resample(X_train, y_train)
    return X_resampled, y_resampled

12 combinations without tuning

In [5]:
perc_col, lag_col, oversampling_method, accuracy, precision, recall, f1, pred_count = [], [], [], [], [], [], [], []

for perc in ['3perc', '5perc']:
    for lag in [3, 7]:
        for oversampling in ['smote', 'random_oversampler', 'adasyn']:
            X_train = eval(f'X_train_{perc}_lag{lag}')
            X_test = eval(f'X_test_{perc}_lag{lag}')
            y_train = eval(f'y_train_{perc}_lag{lag}')
            y_test = eval(f'y_test_{perc}_lag{lag}')

            # oversampling
            if oversampling == 'smote':
                X_train_oversampled, y_train_oversampled = smote(X_train, y_train)
            elif oversampling == 'random_oversampler':
                X_train_oversampled, y_train_oversampled = random_oversampler(X_train, y_train)
            else:
                X_train_oversampled, y_train_oversampled = adasyn(X_train, y_train)

            # scaling for Naive Bayes
            scaler = MinMaxScaler()
            X_train_oversampled = scaler.fit_transform(X_train_oversampled)
            X_test = scaler.transform(X_test)
            
            # fit and predict
            nb = MultinomialNB()
            pred = nb.fit(X_train_oversampled, y_train_oversampled).predict(X_test)

            # update columns
            perc_col.append(perc)
            lag_col.append(lag)
            oversampling_method.append(oversampling)
            accuracy.append(accuracy_score(y_test, pred))
            precision.append(precision_score(y_test, pred, average='weighted'))
            recall.append(recall_score(y_test, pred, average='weighted'))
            f1.append(f1_score(y_test, pred, average='weighted'))
            pred_count.append(dict(pd.Series(pred).value_counts().sort_index()))

results_no_tuning = pd.DataFrame({
    'model': "NB",
    'perc_threshold': perc_col,
    'lag': lag_col,
    'oversampling': oversampling_method,
    'accuracy': accuracy,
    'precision': precision,
    'recall': recall,
    'f1': f1,
    'pred_count': pred_count
})
results_no_tuning

,model,perc_threshold,lag,oversampling,accuracy,precision,recall,f1,pred_count
0,NB,3perc,3,smote,0.470238,0.394611,0.470238,0.410436,"{-1: 40, 0: 122, 1: 6}"
1,NB,3perc,3,random_oversampler,0.446429,0.386523,0.446429,0.401967,"{-1: 39, 0: 117, 1: 12}"
2,NB,3perc,3,adasyn,0.488095,0.369860,0.488095,0.419740,"{-1: 47, 0: 120, 1: 1}"
3,NB,3perc,7,smote,0.458333,0.395025,0.458333,0.403149,"{-1: 45, 0: 118, 1: 5}"
4,NB,3perc,7,random_oversampler,0.458333,0.414289,0.458333,0.409659,"{-1: 43, 0: 118, 1: 7}"
5,NB,3perc,7,adasyn,0.482143,0.367125,0.482143,0.416620,"{-1: 52, 0: 116}"
6,NB,5perc,3,smote,0.720238,0.687208,0.720238,0.703244,"{-1: 25, 0: 143}"
7,NB,5perc,3,random_oversampler,0.744048,0.691707,0.744048,0.716830,"{-1: 20, 0: 148}"
8,NB,5perc,3,adasyn,0.726190,0.685368,0.726190,0.705183,"{-1: 23, 0: 145}"
9,NB,5perc,7,smote,0.755952,0.703467,0.755952,0.728650,"{-1: 20, 0: 148}"


Tuning for 5perc, lag7 and adasyn

In [6]:
perc_col, lag_col, oversampling_method, param, accuracy, precision, recall, f1 = [], [], [], [], [], [], [], []

# best perc, lag and oversampling method
perc = '5perc'
lag = 7
oversampling = 'adasyn'
X_train = eval(f'X_train_{perc}_lag{lag}')
X_test = eval(f'X_test_{perc}_lag{lag}')
y_train = eval(f'y_train_{perc}_lag{lag}')
y_test = eval(f'y_test_{perc}_lag{lag}')

# oversampling
if oversampling == 'smote':
    X_train_oversampled, y_train_oversampled = smote(X_train, y_train)
elif oversampling == 'random_oversampler':
    X_train_oversampled, y_train_oversampled = random_oversampler(X_train, y_train)
else:
    X_train_oversampled, y_train_oversampled = adasyn(X_train, y_train)

# scaling for Naive Bayes
scaler = MinMaxScaler()
X_train_oversampled = scaler.fit_transform(X_train_oversampled)
X_test = scaler.transform(X_test)

# tuning
nb = MultinomialNB()
lgbm_clf = RandomizedSearchCV(nb, nb_distribution, n_iter=100, scoring=['accuracy', 'recall_weighted', 'precision_weighted', 'f1_weighted'], refit='f1_weighted', random_state=42)
lgbm_search = lgbm_clf.fit(X_train_oversampled, y_train_oversampled)

# update columns
perc_col.append([perc]*100)
lag_col.append([lag]*100)
oversampling_method.append([oversampling]*100)
param.append(lgbm_search.cv_results_['params'])
accuracy.append(lgbm_search.cv_results_['mean_test_accuracy'])
precision.append(lgbm_search.cv_results_['mean_test_precision_weighted'])
recall.append(lgbm_search.cv_results_['mean_test_recall_weighted'])
f1.append(lgbm_search.cv_results_['mean_test_f1_weighted'])

In [7]:
results = pd.DataFrame({
    'model': "NB",
    'perc_threshold': np.array(perc_col).flatten(),
    'lag': np.array(lag_col).flatten(),
    'oversampling': np.array(oversampling_method).flatten(),
    'parameters': np.array(param).flatten(),
    'accuracy': np.array(accuracy).flatten(),
    'precision': np.array(precision).flatten(),
    'recall': np.array(recall).flatten(),
    'f1': np.array(f1).flatten(),
}).sort_values(by="f1", ascending=False).reset_index(drop=True)

In [8]:
results.head(10)

,model,perc_threshold,lag,oversampling,parameters,accuracy,precision,recall,f1
0,NB,5perc,7,adasyn,{'alpha': 85.98737339212275},0.673122,0.738358,0.673122,0.643717
1,NB,5perc,7,adasyn,{'alpha': 70.45683638454503},0.671729,0.737399,0.671729,0.642117
2,NB,5perc,7,adasyn,{'alpha': 70.72114131472235},0.671729,0.737399,0.671729,0.642117
3,NB,5perc,7,adasyn,{'alpha': 67.32248920775338},0.671496,0.737277,0.671496,0.641981
4,NB,5perc,7,adasyn,{'alpha': 56.69849511478852},0.670102,0.736149,0.670102,0.640930
5,NB,5perc,7,adasyn,{'alpha': 55.517216852447234},0.669869,0.735960,0.669869,0.640642
6,NB,5perc,7,adasyn,{'alpha': 49.83043837494908},0.668706,0.735199,0.668706,0.639488
7,NB,5perc,7,adasyn,{'alpha': 40.679084943595456},0.666847,0.734030,0.666847,0.637282
8,NB,5perc,7,adasyn,{'alpha': 35.20481045526037},0.666150,0.733274,0.666150,0.636318
9,NB,5perc,7,adasyn,{'alpha': 29.794544625913627},0.664755,0.732017,0.664755,0.634936


Best Model

In [9]:
best_params, best_perc, best_lag, best_oversampling = results.iloc[0]['parameters'], results.iloc[0]['perc_threshold'], results.iloc[0]['lag'], results.iloc[0]['oversampling']
best_model = MultinomialNB(**best_params)
X_train, y_train, x_test, y_test = eval(f'X_train_{best_perc}_lag{best_lag}'), eval(f'y_train_{best_perc}_lag{best_lag}'), eval(f'X_test_{best_perc}_lag{best_lag}'), eval(f'y_test_{best_perc}_lag{best_lag}')

# scaling for Naive Bayes
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

if best_oversampling == 'smote':
    X_train_oversampled, y_train_oversampled = smote(X_train, y_train)
elif best_oversampling == 'random_oversampler':
    X_train_oversampled, y_train_oversampled = random_oversampler(X_train, y_train)
else:
    X_train_oversampled, y_train_oversampled = adasyn(X_train, y_train)

best_model.fit(X_train_oversampled, y_train_oversampled)
best_model

MultinomialNB(alpha=85.98737339212275)

In [10]:
def print_results(actual, predictions):
    print(f"accuracy: {accuracy_score(actual, predictions)}")
    print(f"precision: {precision_score(actual, predictions, average='weighted')}")
    print(f"recall: {recall_score(actual, predictions, average='weighted')}")
    print(f"f1: {f1_score(actual, predictions, average='weighted')}")
    print(f"confusion matrix:\n{confusion_matrix(actual, predictions)}")

Predict on test set

In [11]:
test_pred = best_model.predict(X_test)
print_results(y_test, test_pred)

accuracy: 0.8154761904761905
precision: 0.6650014172335601
recall: 0.8154761904761905
f1: 0.7325917252146761
confusion matrix:
[[  0  21   0]
 [  0 137   0]
 [  0  10   0]]


Predict on full data

In [12]:
X_full = np.vstack([X_train, X_test])
y_full = pd.concat([y_train, y_test])

full_pred = best_model.predict(X_full)
print_results(y_full, full_pred)

accuracy: 0.6700416914830256
precision: 0.9135879820278063
recall: 0.6700416914830256
f1: 0.7575919429438953
confusion matrix:
[[  42   22    2]
 [ 313 1065  195]
 [   9   13   18]]


Business Metric

In [22]:
X_train_df = pd.DataFrame(data=X_train, columns=X_train_3perc_lag3.columns, index=X_train_3perc_lag3.index)
X_test_df = pd.DataFrame(data=X_test, columns=X_test_3perc_lag3.columns, index=X_test_3perc_lag3.index)

In [24]:
whole_df_x = pd.concat([X_train_df, X_test_df])
whole_df_x.shape

(1679, 11)

In [25]:
whole_df_x['year'] = whole_df_x.index.year

In [26]:
business_metric_results = pd.DataFrame(columns=['exp_strategy_annual_return', 'exp_benchmark_annual_return'])

for year in [2016, 2017, 2018, 2019, 2020, 2021, 2022]:
    year_data = whole_df_x[whole_df_x['year'] == year]
    # year_data = year_data.set_index('index')
    year_data = year_data.drop(['year'], axis = 1)
    predict_x = best_model.predict(np.array(year_data)) 
    # predictions = np.argmax(predict_x,axis=1)
    predictions = predict_x

    df_pred = pd.DataFrame({'prediction':predictions}, index=year_data.index)
    df_pred = df_pred.replace({-1:1, 1:-1}) # convert classes to buy hold sell
    dates = df_pred.index

    if year == 2022:
        end_date = "2022-09-02"
    else:
        end_date = str(year+1) + "-01-01"
    df_prices = yf.download("^GSPC", start=dates[0], end=end_date)[['Adj Close']]

    # create positions column
    positions = []
    prev = 0
    for i in range(len(df_pred)):
        if df_pred.iloc[i]['prediction'] == 0:
            positions.append(prev)
        else:
            prev = df_pred.iloc[i]['prediction']
            positions.append(prev)

    df_business = pd.DataFrame()
    df_business['stock_daily_log_return'] = np.log(df_prices /df_prices.shift(1))['Adj Close']
    df_business['prediction'] = df_pred['prediction']
    df_business['position'] = positions
    df_business['benchmark'] = 1 # long and hold strategy
    df_business["strategy_Returns"] = df_business["stock_daily_log_return"] * df_business["position"].shift(1)
    df_business["benchmark_Returns"] = df_business["stock_daily_log_return"] * df_business["benchmark"].shift(1)

    # Annual Mean Returns or Expected returns
    expected_strategy_annual_return = np.exp(df_business['strategy_Returns'].mean() * 252) - 1 
    expected_benchmark_annual_return = np.exp(df_business['benchmark_Returns'].mean() * 252) - 1 
    business_metric_results.loc[year] = [expected_strategy_annual_return, expected_benchmark_annual_return]
    # print(f'Expected Annual Returns: Strategy: {round(expected_strategy_annual_return*100, 2)}%  |  Stock: {round(expected_benchmark_annual_return*100, 2)}%')

business_metric_results

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,exp_strategy_annual_return,exp_benchmark_annual_return
2016,0.042123,0.112846
2017,0.131417,0.185753
2018,-0.093125,-0.070634
2019,0.288443,0.288443
2020,-0.499854,0.152929
2021,-0.224343,0.289230
2022,0.000000,-0.249185
